# Testing custom CNN trained on padded and translated MNIST train set on affNIST test set

In [1]:
from keras import backend as K
from keras.datasets import mnist
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Conv2D, Activation, Dense, Dropout, Lambda
from keras.layers import BatchNormalization, MaxPooling2D, Flatten, Conv1D
from keras.utils import to_categorical
from keras.callbacks import Callback, ModelCheckpoint, TensorBoard
from keras import optimizers
from keras import regularizers
from keras import losses
import numpy as np
from scipy.io import matlab
import os

Using TensorFlow backend.


## Load train set generated in generate_datasets.py

In [2]:
img_rows, img_cols = 40, 40
num_classes = 10

(t_x_train, t_y_train), _ = mnist.load_data()

reps = 1

t_x_train = np.repeat(t_x_train, reps, axis=0)
x_train = np.zeros((t_x_train.shape[0], 40, 40), dtype=np.float32)

for i in range(0, x_train.shape[0]):
    x, y = np.random.randint(0, 12, 2)
    x_train[i, y:y+28, x:x+28] = t_x_train[i]

y_train = np.repeat(t_y_train, reps, axis=0)

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_train /= 255.0

y_train = to_categorical(y_train, num_classes)

input_shape = (img_rows, img_cols, 1)

## Assemble CNN with 3 conv layers and two dense layers

In [3]:
l2 = regularizers.l2(l=0.001)

inp = Input(shape=input_shape)
l = inp

l = Conv2D(8, (3, 3), activation='relu', kernel_regularizer=l2)(l)
l = BatchNormalization()(l)
l = MaxPooling2D((2, 2))(l)
l = Conv2D(16, (3, 3), activation='relu', kernel_regularizer=l2)(l) 
l = BatchNormalization()(l)
l = MaxPooling2D((2, 2))(l)
l = Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2)(l)
l = BatchNormalization()(l)
l = MaxPooling2D((2, 2))(l)
l = Flatten()(l)
l = Dense(72, activation='relu', kernel_regularizer=l2)(l)
l = BatchNormalization()(l)
l = Dense(10, activation='softmax', kernel_regularizer=l2)(l)

model = Model(inputs=inp, outputs=l, name='40x40_input_cnn')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40, 40, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 38, 38, 8)         80        
_________________________________________________________________
batch_normalization_1 (Batch (None, 38, 38, 8)         32        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 19, 19, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 16)        1168      
_________________________________________________________________
batch_normalization_2 (Batch (None, 17, 17, 16)        64        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 16)          0         
__________

In [4]:
def caps_objective(y_true, y_pred):
    return K.sum(y_true * K.clip(0.8 - y_pred, 0, 1) ** 2 + 0.5 * (1 - y_true) * K.clip(y_pred - 0.3, 0, 1) ** 2)

optimizer = optimizers.Adam(lr=0.001)
model.compile(loss=losses.categorical_crossentropy, #caps_objective,
              optimizer=optimizer,
              metrics=['accuracy'])

In [5]:
w_path = os.path.join('weights', model.name)

if not os.path.exists(w_path):
    os.makedirs(w_path)
    
f_name = os.path.join(w_path, 'weights.{epoch:02d}.hdf5')    

In [6]:
# model.load_weights('weights/40x40_input_cnn/weights.03.hdf5')

# Train for 4 epochs

In [9]:
batch_size = 32
num_epochs = 32

h = model.fit(x_train, y_train,
          batch_size=batch_size, epochs=num_epochs, initial_epoch=0,
          verbose=1, validation_split=0.09,
          callbacks=[ModelCheckpoint(f_name)])

Train on 54600 samples, validate on 5400 samples
Epoch 1/32
54600/54600 [==============================] - 47s - loss: 0.1525 - acc: 0.9767 - val_loss: 0.1390 - val_acc: 0.9809
Epoch 2/32
54600/54600 [==============================] - 48s - loss: 0.1446 - acc: 0.9787 - val_loss: 0.2081 - val_acc: 0.9598
Epoch 3/32
54600/54600 [==============================] - 45s - loss: 0.1370 - acc: 0.9792 - val_loss: 0.1397 - val_acc: 0.9789
Epoch 4/32
54600/54600 [==============================] - 45s - loss: 0.1328 - acc: 0.9808 - val_loss: 0.1566 - val_acc: 0.9739
Epoch 5/32
54600/54600 [==============================] - 45s - loss: 0.1291 - acc: 0.9810 - val_loss: 0.1996 - val_acc: 0.9570
Epoch 6/32
54600/54600 [==============================] - 46s - loss: 0.1270 - acc: 0.9818 - val_loss: 0.1349 - val_acc: 0.9785
Epoch 7/32
54600/54600 [==============================] - 46s - loss: 0.1240 - acc: 0.9812 - val_loss: 0.1260 - val_acc: 0.9831
Epoch 8/32
54600/54600 [==============================]

## Test on [affNIST test set](http://www.cs.toronto.edu/~tijmen/affNIST/32x/transformed/test.mat.zip)

In [10]:
f = matlab.loadmat('affnist/test.mat')

x_test = f['affNISTdata'][0, 0][2].astype(np.float32).transpose()
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
x_test /= 255
input_shape = (img_rows, img_cols, 1)

y_test = f['affNISTdata'][0, 0][4].transpose()

score = model.evaluate(x_test, y_test, verbose=1)
print('')
print('Test score: ', score[0])
print('Test accuracy: ', score[1])    

319904/320000 [============================>.] - ETA: 0s
Test score:  1.01800707984
Test accuracy:  0.74180625
